In [164]:
import pandas as pd
import altair as alt


cd = pd.read_csv("Crime_Data_from_2020_to_Present (1).csv")


cd = cd[['Crm Cd Desc', 'TIME OCC', 'LON', 'LAT']].dropna()

cd = cd[(cd['LAT'] != 0) & (cd['LON'] != 0)] 

cd.head()

,Crm Cd Desc,TIME OCC,LON,LAT
0,VEHICLE - STOLEN,2130,-118.3506,34.0375
1,BURGLARY FROM VEHICLE,1800,-118.2628,34.0444
2,BIKE - STOLEN,1700,-118.3002,34.0210
3,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),2037,-118.4387,34.1576
4,THEFT OF IDENTITY,1200,-118.3277,34.0944


In [165]:
cd_subset = cd.sample(n=4900, random_state=42) #choosing a random subset of data

In [166]:
cd_subset.shape

(4900, 4)

In [167]:
cd_subset['HOUR'] = cd_subset['TIME OCC'] // 100 #creates a column "HOUR" by dividing all the columns in "TIME OCC" by 100

crime_type_dropdown = alt.binding_select(
    options=cd_subset['Crm Cd Desc'].unique().tolist(),
    name='Crime Type: '
) #creates a dropdown menu of unique crime types

crime_type_selection = alt.selection_point(fields=['Crm Cd Desc'], bind=crime_type_dropdown, value=cd_subset['Crm Cd Desc'].unique()[0])


In [168]:
scatter_plot = alt.Chart(cd_subset).mark_circle(size=50, color='blue').encode(
    x=alt.X('LON:Q', title='Longitude', scale=alt.Scale(domain=[-119, -118])), #restricting the latitude and longitude to LA area
    y=alt.Y('LAT:Q', title='Latitude', scale=alt.Scale(domain=[33.5, 34.5])),
    tooltip=['Crm Cd Desc', 'LAT', 'LON']
).add_params(
    crime_type_selection #adding the crime_type_selection filter from above
).transform_filter(
    crime_type_selection
).properties(
    title="Crime Locations by Latitude and Longitude",
    width=600,
    height=400
)

In [169]:
histogram = alt.Chart(cd_subset).mark_bar(color='red').encode( #creating a red bar chart for time crime distribution
    x=alt.X('HOUR:Q', bin=alt.Bin(maxbins=24, extent=[0, 24]), title='Hour of the Day'), #24 bins because there are 24 hours in the day
    y=alt.Y('count()', title='Count'),
    tooltip=['count()']
).transform_filter(
    crime_type_selection
).properties(
    title="Time Distribution of Crimes",
    width=600,
    height=400
)

In [170]:
dashboard = alt.vconcat( #vertically concatenates the two graphs
    scatter_plot,
    histogram,
    spacing=50
).configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=14
)

In [171]:
dashboard 

alt.VConcatChart(...)

In [172]:
myJekyllDir = './'


In [173]:
dashboard.save(myJekyllDir + 'LA_crime_dashboard1.json')